## Initialization

In [ ]:
%reload_ext autoreload
%autoreload 2
%reload_ext cython
%reload_ext line_profiler
import os,sys
sys.path.insert(1, os.path.join(sys.path[0], '..', 'module'))
import wiki
import numpy as np
import pandas as pd
import networkx as nx
import scipy as sp
import seaborn as sns
import cufflinks as cf
import matplotlib.pyplot as plt

In [ ]:
# topics = ['anatomy', 'biochemistry', 'cognitive science', 'evolutionary biology',
#           'genetics', 'immunology', 'molecular biology', 'chemistry', 'biophysics',
#           'energy', 'optics', 'earth science', 'geology', 'meteorology']
topics = ['earth science']
path_saved = '/Users/harangju/Developer/data/wiki/graphs/dated/'
networks = {}
for topic in topics:
    print(topic, end=' ')
    networks[topic] = wiki.Net()
    networks[topic].load_graph(path_saved + topic + '.pickle')
graph = networks[topic].graph

In [ ]:
topic = topics[0]
graph = networks[topic].graph
tfidf = graph.graph['tfidf']

In [ ]:
import pickle
dct = pickle.load(open('/Users/harangju/Developer/data/wiki/models/' + 'dict.model','rb'))

## Auxiliary methods

In [ ]:
%%cython -f

import numpy as np
cimport numpy as np
from cython cimport floating,boundscheck,wraparound
from cython.parallel import prange

from libc.math cimport fabs

np.import_array()

@boundscheck(False)  # Deactivate bounds checking
@wraparound(False)
def cython_manhattan(floating[::1] X_data, int[:] X_indices, int[:] X_indptr,
                     floating[::1] Y_data, int[:] Y_indices, int[:] Y_indptr,
                     double[:, ::1] D):
    """Pairwise L1 distances for CSR matrices.
    Usage:
    >>> D = np.zeros(X.shape[0], Y.shape[0])
    >>> cython_manhattan(X.data, X.indices, X.indptr,
    ...                  Y.data, Y.indices, Y.indptr,
    ...                  D)
    """
    cdef np.npy_intp px, py, i, j, ix, iy
    cdef double d = 0.0
    
    cdef int m = D.shape[0]
    cdef int n = D.shape[1]
    
    with nogil:                          
        for px in prange(m):
            for py in range(n):
                i = X_indptr[px]
                j = Y_indptr[py]
                d = 0.0
                while i < X_indptr[px+1] and j < Y_indptr[py+1]:
                    if i < X_indptr[px+1]: ix = X_indices[i]
                    if j < Y_indptr[py+1]: iy = Y_indices[j]
                    
                    if ix==iy:
                        d = d+fabs(X_data[i]-Y_data[j])
                        i = i+1
                        j = j+1
                    
                    elif ix<iy:
                        d = d+fabs(X_data[i])
                        i = i+1
                    else:
                        d = d+fabs(Y_data[j])
                        j = j+1
                
                if i== X_indptr[px+1]:
                    while j < Y_indptr[py+1]:
                        iy = Y_indices[j]
                        d = d+fabs(Y_data[j])
                        j = j+1                                            
                else:
                    while i < X_indptr[px+1]:
                        ix = X_indices[i]
                        d = d+fabs(X_data[i])
                        i = i+1
                        
                D[px,py] = d

In [ ]:
import sklearn.preprocessing as skp
import sklearn.metrics.pairwise as smp

def year_diffs(graph):
    return [graph.nodes[node]['year'] - graph.nodes[neighbor]['year']
            for node in graph.nodes
            for neighbor in list(graph.successors(node))]

def neighbor_similarity(graph, tfidf):
    nodes = list(graph.nodes)
    return [smp.cosine_similarity(tfidf[:,nodes.index(node)].transpose(),
                                  tfidf[:,nodes.index(neighbor)].transpose())[0,0]
            for node in nodes
            for neighbor in list(graph.successors(node))]

def parent_similarity(graph, tfidf):
    nodes = list(graph.nodes)
    return [smp.cosine_similarity(tfidf[:,nodes.index(node)].transpose(),
                                  tfidf[:,nodes.index(neighbor)].transpose())[0,0]
            for node in nodes
            for neighbor in list(graph.predecessors(node))]

def non_neighbor_similarity(graph, tfidf):
    nodes = list(graph.nodes)
    sim = [smp.cosine_similarity(tfidf[:,nodes.index(n1)].transpose(),
                                 tfidf[:,nodes.index(n2)].transpose())[0,0]
           for n1 in graph.nodes
           for n2 in graph.nodes
           if (n2 is not n1) and (n2 not in list(graph.neighbors(n1)))]
    return sim

def sparse_manhattan(X,Y=None):
    X, Y = smp.check_pairwise_arrays(X, Y)
    X = sp.sparse.csr_matrix(X, copy=False)
    Y = sp.sparse.csr_matrix(Y, copy=False)
    res = np.empty(shape=(X.shape[0],Y.shape[0]))
    cython_manhattan(X.data,X.indices,X.indptr,
                     Y.data,Y.indices,Y.indptr,
                             res)
    return res

def word_diffs(graph, tfidf):
    dists = sparse_manhattan(X=skp.binarize(tfidf).transpose())
    nodes = list(graph.nodes)
    return [dists[nodes.index(node), nodes.index(neighbor)]
            for node in nodes
            for neighbor in list(graph.successors(node))]

def sum_weight_differences(graph, tfidf):
    nodes = list(graph.nodes)
    diff = []
    for node in nodes:
        for neighbor in graph.successors(node):
            v1 = tfidf[:,nodes.index(node)]
            v2 = tfidf[:,nodes.index(neighbor)]
            idx = np.concatenate([v1.indices, v2.indices])
            diff.append( np.sum(np.absolute(v1[idx]-v2[idx])) )
    return diff

def weight_differences(graph, tfidf):
    nodes = list(graph.nodes)
    diff = np.empty(shape=0)
    for node in nodes:
        for neighbor in graph.successors(node):
            v1 = tfidf[:,nodes.index(node)]
            v2 = tfidf[:,nodes.index(neighbor)]
            idx = np.concatenate([v1.indices, v2.indices])
            diff = np.append(diff, np.absolute(v1[idx]-v2[idx]).data)
    return diff

def plot_distribution(data):
    bins = np.logspace(np.log10(min(data)), np.log10(max(data)), 30)
    hist, edges = np.histogram(data, bins=bins)
#     hist_norm = hist/(bins[1:] - bins[:-1])
    sns.scatterplot(bins[:-1], hist/len(data))
    plt.yscale('log')
    plt.xscale('log')
    plt.xlim(bins[0]/2, bins[-1]*2)
    plt.ylim(min(hist[hist>0])/len(data)/2, 1)
    plt.xlabel('x')
    plt.ylabel('P(x)')

## Priors

### Prior: power law distributions of weights

In [ ]:
import powerlaw
tfidf = graph.graph['tfidf']
fit = powerlaw.Fit(tfidf.data)
# fit = xmin = 4.3e-2; alpha=2.7

n_rows = 2
plt.figure(figsize=(16,n_rows*6))

plt.subplot(n_rows,2,1)
fit.plot_pdf()
fit.power_law.plot_pdf();
plt.title(f"xmin={fit.xmin:.1e}, α={fit.alpha:.1f}");

plt.subplot(n_rows,2,3)
sns.scatterplot(x='index', y='weight',
                data=pd.DataFrame({'index': tfidf.indices,
                                   'weight': tfidf.data}))
sns.scatterplot(x='index', y='weight',
                data=pd.DataFrame({'index': tfidf.indices,
                                   'weight': tfidf.data})\
                       .groupby('index').mean()\
                       .reset_index())
plt.legend(['weights', 'averaged'])
plt.ylim([-.2,1.2])

plt.subplot(n_rows,2,4)
plot_distribution(tfidf.data)

### Prior: similarity / year between neighbors

In [ ]:
n_rows = 5

plt.figure(figsize=(16,n_rows*6))
plt.subplot(n_rows,2,1)
yd = year_diffs(graph)
sns.distplot(yd)
plt.xlabel('Δyear')
plt.ylabel('distribution')

plt.subplot(n_rows,2,2)
bin_size=25
years = [graph.nodes[node]['year'] for node in graph.nodes]
sns.distplot(years, bins=bin_size, rug=True, kde=False)
hist, bin_edges = np.histogram(years, bins=bin_size)
popt, pcov = sp.optimize.curve_fit(lambda x,a,b: a*pow(b,x), bin_edges[1:], hist)
x = np.linspace(min(years), max(years), 100)
sns.lineplot(x=x, y=popt[0]*pow(popt[1],x))
plt.legend([f"a*b^x; a={popt[0]:.1e}, b={popt[1]:.4f}"])
plt.xlabel('year');

wd = word_diffs(graph, tfidf)
mu, std = sp.stats.norm.fit(wd)
plt.subplot(n_rows,2,3)
sns.distplot(wd)
x = np.linspace(min(wd), max(wd), 100)
plt.plot(x, sp.stats.norm.pdf(x, mu, std))
plt.legend([f"m={mu:.2f}; s={std:.2f}"])
plt.xlabel('manhattan distance')
plt.ylabel('distribution');

slope, intercept, fit_r, p, stderr = sp.stats.linregress(np.abs(yd), wd)
plt.subplot(n_rows,2,4)
wd = word_diffs(graph, tfidf)
sns.scatterplot(x=np.abs(yd), y=wd)
x = np.linspace(0, max(yd), 100)
sns.lineplot(x, np.multiply(slope, x) + intercept)
plt.title(f"r={fit_r:.2f}; p={p:.1e}")
plt.legend([f"slope={slope:.2f}"])
plt.xlabel('Δyear')
plt.ylabel('manhattan distance');

parents = parent_similarity(graph, tfidf)
neighbors = neighbor_similarity(graph, tfidf)
non_neighbors = non_neighbor_similarity(graph, tfidf)
fit_mu, fit_std = sp.stats.norm.fit(neighbors)
plt.subplot(n_rows,2,5)
sns.distplot(neighbors, hist=True)
x = np.linspace(min(neighbors), max(neighbors), 100)
plt.plot(x, sp.stats.norm.pdf(x, fit_mu, fit_std))
sns.distplot(non_neighbors)
plt.legend([f"fit-neighbors (m={fit_mu:.2f}; s={fit_std:.2f})", 'neighbors', 'non-neighbors'])
plt.xlabel('cos similarity');

slope, intercept, r, p, stderr = sp.stats.linregress(np.abs(yd), neighbors)
plt.subplot(n_rows,2,6)
sns.scatterplot(x=np.abs(yd), y=neighbors)
x = np.linspace(0, max(yd), 100)
sns.lineplot(x, np.multiply(slope, x) + intercept)
plt.title(f"r={r:.2f}; p={p:.1f}")
plt.legend([f"slope={slope:.2f}"])
plt.xlabel('Δyear')
plt.ylabel('cosine similarity');

sum_weight_diffs = sum_weight_differences(graph, tfidf)
mu, std = sp.stats.norm.fit(sum_weight_diffs)
plt.subplot(n_rows,2,7)
sns.distplot(sum_weight_diffs)
x = np.linspace(min(sum_weight_diffs), max(sum_weight_diffs), 100)
plt.plot(x, sp.stats.norm.pdf(x, mu, std))
plt.legend([f"m={mu:.2f}; s={std:.2f}"])
plt.xlabel('Σ abs Δw_i')
plt.ylabel('distribution');

slope, intercept, fit_r_sum_weight, p, stderr = \
    sp.stats.linregress(np.abs(yd), sum_weight_diffs)
plt.subplot(n_rows,2,8)
sns.scatterplot(x=np.abs(yd), y=sum_weight_diffs)
x = np.linspace(0, max(yd), 100)
sns.lineplot(x, np.multiply(slope, x) + intercept)
plt.title(f"r={fit_r_sum_weight:.2f}; p={p:.1e}")
plt.legend([f"slope={slope:.1e}"])
plt.xlabel('Δyear')
plt.ylabel('Σ abs Δw_i');

weight_diffs = weight_differences(graph, tfidf)
fit_mu_weight, std = sp.stats.norm.fit(weight_diffs)
plt.subplot(n_rows,2,9)
sns.distplot(weight_diffs)
x = np.linspace(min(weight_diffs), max(weight_diffs), 100)
plt.plot(x, sp.stats.norm.pdf(x, mu, std))
plt.legend([f"m={fit_mu_weight:.2f}; s={std:.2f}"])
plt.xlabel('abs Δw_i')
plt.ylabel('distribution');

Maybe this just shows that people nowadays have a lower threshold for dissimilarity between old and new knowledge.

### Prior: word weight vs title

In [ ]:
stoplist=set('for a of the and to in'.split())
nodes = []
words = []
for i in range(tfidf.shape[1]):
    node = list(graph.nodes)[i]
    if tfidf[:,i].data.size == 0:
        print(node, tfidf[:,i].data)
        continue
    top_words, idx = wiki.Model.find_top_words(tfidf[:,i], dct, top_n=5)
    nodes += [node]
    words += [top_words]
pd.DataFrame(data={'Node': nodes, 'Top words': words})

## Static methods

### Power law

In [ ]:
max_val = np.max(tfidf.data)
def f(n, max_val):
    x = fit.power_law.generate_random(n)
    while True:
        if np.any(x>max_val):
            y = x > max_val
            x[y] = fit.power_law.generate_random(np.sum(y))
        else:
            break
    return x

g = lambda n: np.random.choice(tfidf.data, size=n)

plt.figure(figsize=(16,6))

plt.subplot(121)
plot_distribution(tfidf.data)
plot_distribution(f(100000, max_val))
plot_distribution(g(10000))
plt.xlim([1e-4,1])
plt.ylim([1e-5,1])
# plt.yscale('linear')
# plt.xscale('linear')
plt.title('weights');

plt.subplot(122)
sns.distplot(tfidf.data[tfidf.data<fit.xmin])
plt.xlabel('weight')
plt.ylabel('count');

### Mutate

In [ ]:
x = tfidf[:,0].copy()
y = tfidf[:,0].copy()
T = 300

sim = np.zeros(T)
size = np.zeros(T)
mag = np.zeros(T)
for i in range(sim.size):
    sim[i] = smp.cosine_similarity(x.transpose(),y.transpose())[0,0]
    size[i] = y.size
    mag[i] = np.sum(y.data)
    y = wiki.Model.mutate(y, lambda n: fit.power_law.generate_random(n),
                          point=(1,.5), insert=(1,.3,None), delete=(1,.3))

plt.figure(figsize=(16,10))
ax = plt.subplot(221)
sns.lineplot(x=range(sim.size), y=sim)
plt.ylabel('similarity')
plt.xlabel('years')

plt.subplot(222)
sns.lineplot(x=range(sim.size), y=size)
plt.ylabel('size')
plt.xlabel('years')

plt.subplot(223)
plot_distribution(x.data)
plot_distribution(y.data)
plt.xlabel('tf-idf values')
plt.legend(['before mutation', 'after mutation'])
plt.xlabel('tf-idf values')

plt.subplot(224)
plot_distribution(x.data)
plot_distribution(y.data)
plt.xlabel('tf-idf values')
plt.yscale('linear')
plt.xscale('linear')
plt.ylim([0,.2])
plt.xlim([0,.1])
plt.legend(['before mutation','after mutation']);

### Connect

In [ ]:
model = wiki.Model(graph_parent=networks[topic].graph,
                   vectors_parent=networks[topic].graph.graph['tfidf'],
                   year_start=-500)

In [ ]:
test_graph = model.graph.copy()
test_vector = sp.sparse.hstack([tfidf[:,list(graph.nodes).index(n)] for n in test_graph.nodes])

seed = 'Meteorology'
seed_vector = tfidf[:,list(graph.nodes).index(seed)]

print('Nodes:', test_graph.nodes)
print('Edges:', test_graph.edges, '\n')
print(f"Seed: {seed}\n")
wiki.Model.connect(seed_vector, test_graph, test_vector, dct, match_n=3)
print('Nodes:', test_graph.nodes)
print('Edges:', test_graph.edges)

## Evolve

### Model

In [ ]:
model = wiki.Model(graph_parent=networks[topic].graph,
                   vectors_parent=networks[topic].graph.graph['tfidf'],
                   year_start=1600)
print(f"Topic: '{graph.name}'\n" +\
      f"Core nodes\n" +\
      f"   {list(model.graph.nodes)}\n" +\
      f"Parameters\n" +\
      f"   α (power law): {fit.alpha:.2f}\n" +\
      f"   p_insert/delete: {fit_r:.2f}/2\n" +\
      f"   neighbor_mu, std: {fit_mu:.2f}, {fit_std:.2f}\n" +\
      f"   crossover threshold: {fit_mu+3*fit_std:.2f}\n" +\
      f"   point n={fit_r_sum_weight/fit_mu_weight:.2f};" +\
      f" p={fit_mu_weight:.2f}")

In [ ]:
max_val = np.max(tfidf.data)
%lprun -f model.evolve model.evolve(year_end=1800,\
                                    n_seeds=2,\
                                    point=(fit_r_sum_weight/fit_mu_weight,\
                                           np.mean(tfidf.data)/fit_r_sum_weight),\
                                    insert=(1, fit_r/2, list(set(tfidf.indices))),\
                                    delete=(1, fit_r/2),\
                                    rvs=lambda n: np.vectorize(lambda x: max_val if x>max_val else x,\
                                                               otypes=[np.float64])\
                                                              (fit.power_law.generate_random(n)),\
                                    dct=dct, create=lambda n: np.random.normal(loc=fit_mu,\
                                                                               scale=fit_std, size=n),\
                                    crossover=fit_mu+3*fit_std)
model.record

### Interesting thought
If it weren't for the Middle Ages, we would have an amount of knowledge in the 16th Century that is similar to what we have now. But if we run the model after the Dark Ages, the model is accurate (?).

In [ ]:
s = lambda a,b: smp.cosine_similarity(a.transpose(), b.transpose())[0,0]
nodes = list(model.graph.nodes)
model.record['Similarity to parent'] = [s(model.record.iloc[i]['Seed vectors'],
                                          model.vectors[:,nodes.index(model.record.iloc[i]['Parent'])])
                                        for i in range(len(model.record.index))]
model.record['Parent seed'] = model.record['Parent'] + ' ' + model.record['Seed number'].map(str)

In [ ]:
plt.figure(figsize=(16,10))
ax = sns.lineplot(x='Year', y='Similarity to parent', hue='Parent seed', legend=False,
                  data=model.record)
plt.ylim([0,1.1]);

### Save/load graph

In [ ]:
# nx.write_gpickle(model, f"model6.pickle")

In [ ]:
model = nx.read_gpickle(f"model6.pickle")
model.record

## Posteriors

### Interactive plots
[FigureWidgets](https://plot.ly/python/v3/figurewidget-app/)

In [ ]:
import cufflinks as cf
cf.go_offline()
from ipywidgets import interact, widgets, Layout
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from IPython.display import display

In [ ]:
df = cf.datagen.lines(5,1000).reset_index(drop=True)
df

In [ ]:
def update(change):
    with fig.batch_update():
        fig.data[0].x = df.index[df.index > change.new]
        fig.data[0].y = df[df.columns[0]][df.index > change.new]
min_idx = min(df.index)
max_idx = max(df.index)

slider = widgets.IntSlider(value=0, min=min_idx, max=max_idx,
                           step=1, description='Year', continuous_update=True,
                           layout=Layout(width='auto'))
slider.observe(update, names='value')
display(slider)

fig = go.FigureWidget()
fig.add_trace(go.Scatter(x=df.index, y=df[df.columns[0]], name=df.columns[0], mode='lines'))
fig.update_layout(title='Title', xaxis_title='Index', yaxis_title='Y', template='plotly_white')
fig


### Plots

#### Network growth

In [ ]:
years = pd.DataFrame([model.graph.nodes[node]['year'] for node in model.graph.nodes],
                     columns=['Year'])\
          .sort_values(by='Year')\
          .reset_index(drop=True)
years['count'] = 1
years['Year (cumsum)'] = years['count'].cumsum()
years = years.drop(columns='count')

In [ ]:
nodes = list(model.graph.nodes)
layout = nx.kamada_kawai_layout(model.graph, dim=2)
# layout = nx.spring_layout(model.graph, dim=3)
layout = np.vstack([layout[node] for node in nodes])
Xn = [layout[k][0] for k in range(len(nodes))]
Yn = [layout[k][1] for k in range(len(nodes))]
# Zn = [layout[k][2] for k in range(len(nodes))]
Xe = []
Ye = []
# Ze = []
for e in model.graph.edges:
    Xe += [layout[nodes.index(e[0])][0], layout[nodes.index(e[1])][0], None]
    Ye += [layout[nodes.index(e[0])][1], layout[nodes.index(e[1])][1], None]
#     Ze += [layout[nodes.index(e[0])][2], layout[nodes.index(e[1])][2], None]

In [ ]:
def graph_layout(graph, nodes):
    subgraph = model.graph.subgraph(nodes)
    layout = nx.kamada_kawai_layout(graph, dim=2)
    Xn = [layout[n][0] for n in subgraph.nodes]
    Yn = [layout[n][1] for n in subgraph.nodes]
    Xe = []
    Ye = []
    for e in subgraph.edges:
        Xe += [layout[e[0]][0], layout[e[1]][0], None]
        Ye += [layout[e[0]][1], layout[e[1]][1], None]
    return (Xn, Yn), (Xe, Ye)

In [ ]:
def update_network(change):
    with fig.batch_update():
        (Xn, Yn), (Xe, Ye) = graph_layout(model.graph,
                                          [n for n in model.graph.nodes 
                                           if model.graph.nodes[n]['year']<=change.new])
        fig.data[0].x = Xe
        fig.data[0].y = Ye
        fig.data[1].x = Xn
        fig.data[1].y = Yn
        fig.layout.title = model.graph.name + ', year: ' + str(change.new)
        fig.update_xaxes(range=[-1.2,1.2])
        fig.update_yaxes(range=[-1.2,1.2])

nodes = list(model.graph.nodes)

min_year = min([model.graph.nodes[n]['year'] for n in nodes])
max_year = max([model.graph.nodes[n]['year'] for n in nodes])
slider = widgets.IntSlider(value=min_year, min=min_year, max=max_year,
                           step=1, description='Year', continuous_update=True,
                           layout=Layout(width='auto'))
slider.observe(update_network, names='value')
display(slider)

(Xn, Yn), (Xe, Ye) = graph_layout(model.graph,
                                  [n for n in model.graph.nodes 
                                   if model.graph.nodes[n]['year']==min_year])

trace1 = go.Scatter(x=Xe, y=Ye,# z=Ze, 
                    mode='lines', line=dict(color='gray', width=.5),
                    hoverinfo='none')
trace2 = go.Scatter(x=Xn, y=Yn,# z=Zn, 
                      mode='markers',
                      marker=dict(symbol='circle', size=6,
#                                color=group,
                                  colorscale='Viridis',
                                  line=dict(color='rgb(50,50,50)', width=0.5)),
                      text=nodes, hoverinfo='text')
axis = dict(showbackground=False,
            showline=False,
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            title='')
fig = go.Figure(data=[trace1, trace2],
                layout=go.Layout(title=topic + ', year: ' + str(min_year),
                                 width=600,#1000,
                                 height=600,
                                 showlegend=False,
                                 scene=dict(xaxis=dict(axis),
                                            yaxis=dict(axis),
                                            zaxis=dict(axis),),
                                 hovermode='closest',
                                 template='plotly_white'))
fig = go.FigureWidget(fig)
fig.update_xaxes(range=[-1.2,1.2])
fig.update_yaxes(range=[-1.2,1.2])
fig

In [ ]:
fig = go.Figure(go.Scatter(x=years.Year, y=years['Year (cumsum)'], name='Model'))
fig.update_layout(title='Discoveries', xaxis_title='Year', yaxis_title='Number of discoveries',
                  template='plotly_white')

**Comments**

Too many connections in new nodes. So, try
* restricting title words to uncommon words?

#### Similarity

##### Cosine similarity

In [ ]:
sim = lambda a,b: smp.cosine_similarity(a.transpose(), b.transpose())[0,0]
nodes = list(model.graph.nodes)
model.record['Similarity (parent)'] = [sim(model.record.iloc[i]['Seed vectors'], 
                                  model.vectors[:,nodes.index(model.record.iloc[i]['Parent'])])
                              for i in range(len(model.record.index))]
model.record

In [ ]:
import itertools as it

nodes = list(model.graph.nodes)
births = pd.DataFrame({'Node': nodes,
                       'Year': [model.graph.nodes[n]['year'] for n in nodes]})\
           .sort_values(by=['Year'])\
           .reset_index(drop=True)
births['Similarity (neighbor)'] = [[sim(model.vectors[:,nodes.index(births.iloc[i].Node)],
                                        model.vectors[:,nodes.index(neighbor)])
                                    for neighbor in it.chain(model.graph.successors(births.iloc[i].Node),
                                                             model.graph.predecessors(births.iloc[i].Node))
                                    if model.graph.nodes[neighbor]['year'] <= births.iloc[i].Year]
                                   for i in births.index]
births

In [ ]:
max_y = 180

def update_similarity(change):
    with fig.batch_update():
        fig.data[1].x = [j for i in births[births.Year<=change.new]['Similarity (neighbor)']
                         for j in i]
        fig.data[2].x = model.record['Similarity (parent)'][model.record.Year == change.new]
        fig.update_xaxes(range=[0,1])
        fig.update_yaxes(range=[0,max_y])

min_year = min(model.record.Year)
max_year = max(model.record.Year)
slider = widgets.IntSlider(value=min_year, min=min_year, max=max_year,
                           step=1, description='Year', continuous_update=True,
                           layout=Layout(width='auto'))
slider.observe(update_similarity, names='value')
display(slider)

fig = go.FigureWidget()
fig.add_trace(go.Histogram(x=neighbors,
                           name='empirical'))
fig.add_trace(go.Histogram(x=[j for i in births[births.Year<=min_year+50]['Similarity (neighbor)']
                              for j in i],
                           name='model (neighbor)'))
fig.add_trace(go.Histogram(x=model.record[model.record.Year==min_year]['Similarity (parent)'],
                           name='model (parent)'))
fig.update_layout(title='Cosine similarity', template='plotly_white',
                  xaxis_title='cosine similarity', yaxis_title='number of edges')
fig.update_xaxes(range=[0,1])
fig.update_yaxes(range=[0,max_y])
fig

##### Degree distribution

**Interpretation**

There are too many connections. Similarity to the parent isn't actually changing that much.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=[d for _,d in graph.degree], nbinsx=30, name='empirical'))
fig.add_trace(go.Histogram(x=[d for _,d in model.graph.degree], nbinsx=30, name='model'))
fig.update_layout(title='Degree distribution', template='plotly_white',
                  xaxis_title='degree', yaxis_title='number of edges')

##### Manhattan distance

#### Something

In [ ]:
plt.figure(figsize=(16,4))
plt.subplot(121)
sns.distplot(neighbors)
x = np.linspace(min(neighbors), max(neighbors), 100)
mu, std = sp.stats.norm.fit(neighbors)
plt.plot(x, sp.stats.norm.pdf(x, mu, std))
sns.distplot(non_neighbors)
plt.title(topic + ' (prior)')
plt.legend([f"fit-neighbors (m={mu:.2f}; s={std:.2f})", 'neighbors', 'non-neighbors'])
plt.xlabel('cos similarity');
plt.xlim([-.2,1.2])
plt.subplot(122)
neighbors_model = neighbor_similarity(model.graph, model.vectors)
non_neighbors_model = non_neighbor_similarity(model.graph, model.vectors)
sns.distplot(neighbors_model)
x = np.linspace(min(neighbors_model), max(neighbors_model), 100)
mu, std = sp.stats.norm.fit(neighbors_model)
plt.plot(x, sp.stats.norm.pdf(x, mu, std))
sns.distplot(non_neighbors_model)
plt.title(topic + ' (model)')
plt.legend([f"fit-neighbors (m={mu:.2f}; s={std:.2f})", 'neighbors', 'non-neighbors'])
plt.xlabel('cos similarity')
plt.xlim([-.2,1.2]);

In [ ]:
plt.figure(figsize=(16,4))
plt.subplot(121)
bin_size=25
years = [graph.nodes[node]['year'] for node in graph.nodes]
sns.distplot(years, bins=bin_size, rug=True, kde=False)
hist, bin_edges = np.histogram(years, bins=bin_size)
popt, pcov = sp.optimize.curve_fit(lambda x,a,b: a*pow(b,x), bin_edges[1:], hist)
x = np.linspace(min(years), max(years), 100)
sns.lineplot(x=x, y=popt[0]*pow(popt[1],x))
plt.legend([f"a*b^x; a={popt[0]:.1e}, b={popt[1]:.4f}"])
plt.title('prior')
plt.ylabel('discoveries')
plt.xlabel('year');

plt.subplot(122)
years = [model.graph.nodes[node]['year'] for node in model.graph.nodes]
sns.distplot(years, bins=bin_size, rug=True, kde=False)
hist, bin_edges = np.histogram(years, bins=bin_size)
popt, pcov = sp.optimize.curve_fit(lambda x,a,b: a*pow(b,x), bin_edges[1:], hist)
x = np.linspace(min(years), max(years), 100)
sns.lineplot(x=x, y=popt[0]*pow(popt[1],x))
plt.legend([f"a*b^x; a={popt[0]:.1e}, b={popt[1]:.4f}"])
plt.title('model')
plt.ylabel('discoveries')
plt.xlabel('year');

plt.figure(figsize=(16,4))
bin_size=25
years = [graph.nodes[node]['year'] for node in graph.nodes]
sns.distplot(years, bins=bin_size, rug=True, kde=False, hist=False)
# hist, bin_edges = np.histogram(years, bins=bin_size)
# popt, pcov = sp.optimize.curve_fit(lambda x,a,b: a*pow(b,x), bin_edges[1:], hist)
# x = np.linspace(min(years), max(years), 100)
# sns.lineplot(x=x, y=popt[0]*pow(popt[1],x))
sns.lineplot(x=sorted(years),
             y=np.sum(np.array([sorted(years)]).transpose() < np.array([sorted(years)]), axis=0))

years = [model.graph.nodes[node]['year'] for node in model.graph.nodes]
sns.distplot(years, bins=bin_size, rug=True, kde=False, hist=False)
hist, bin_edges = np.histogram(years, bins=bin_size)
# popt_model, pcov = sp.optimize.curve_fit(lambda x,a,b: a*pow(b,x), bin_edges[1:], hist)
# x = np.linspace(min(years), max(years), 100)
# sns.lineplot(x=x, y=popt_model[0]*pow(popt_model[1],x))
sns.lineplot(x=sorted(years),
             y=np.sum(np.array([sorted(years)]).transpose() < np.array([sorted(years)]), axis=0))

plt.legend([#f"prior: a*b^x; a={popt[0]:.1e}, b={popt[1]:.4f}",
            f"prior: count",
            #f"model: a*b^x; a={popt_model[0]:.1e}, b={popt_model[1]:.4f}",
            f"model: count"])
plt.ylabel('discoveries')
plt.xlabel('year');

In [ ]:
plt.figure(figsize=(16,6))
plt.subplot(121)
fit.plot_pdf()
fit.power_law.plot_pdf()
plt.title(f"empirical xmin={fit.xmin:.1e}, α={fit.alpha:.1f}");
plt.subplot(122)
fit_model = powerlaw.Fit(model.vectors.data)
fit_model.plot_pdf()
fit_model.power_law.plot_pdf()
plt.title(f"model xmin={fit_model.xmin:.1e}, α={fit_model.alpha:.1f}");

In [ ]:
sns.jointplot(x=np.abs(yd), y=wd, kind='reg',
              marginal_kws=dict(bins=15, rug=True))
plt.xlabel('Δyear')
plt.ylabel('manhattan distance');

In [ ]:
n_rows = 4
plt.figure(figsize=(16,n_rows*6))

# wd = word_diffs(graph, tfidf)
# yd = year_diffs(graph)

plt.subplot(n_rows,2,1)
sns.distplot(yd)
plt.title(topic + ' prior')
plt.xlabel('year difference')

plt.subplot(n_rows,2,2)
yd_model = year_diffs(model.graph)
sns.distplot(yd_model)
plt.title(topic + ' model')
plt.xlabel('year difference');

plt.subplot(n_rows,2,3)
sns.scatterplot(x=np.abs(yd), y=wd)
slope, intercept, r, p, stderr = sp.stats.linregress(np.abs(yd), wd)
x = np.linspace(0, max(yd), 100)
sns.lineplot(x, np.multiply(slope, x) + intercept)
plt.title(f"slope={slope:.2f}; r={r:.2f}; p={p:.1e} (prior)")
plt.xlabel('year')
plt.ylabel('manhattan distance');

plt.subplot(n_rows,2,4)
sns.distplot(wd)
mu, std = sp.stats.norm.fit(wd)
x = np.linspace(min(wd), max(wd), 100)
plt.plot(x, sp.stats.norm.pdf(x, mu, std))
plt.xlabel('manhattan distance')
plt.ylabel('probability distribution');
plt.title(f"μ={mu:.2}, σ={std:.2} (prior)")

# wd_model = word_diffs(model.graph, model.vectors)
# yd_model = year_diffs(model.graph)
# neighbors_model = neighbor_similarity(model.graph, model.vectors)

plt.subplot(n_rows,2,5)
sns.scatterplot(x=np.abs(yd_model), y=wd_model)
slope, intercept, r, p, stderr = sp.stats.linregress(np.abs(yd_model), wd_model)
x = np.linspace(0, max(np.abs(yd_model)), 100)
sns.lineplot(x, np.multiply(slope, x) + intercept)
plt.title(f"slope={slope:.2f}; r={r:.2f}; p={p:.1e} (model)")
plt.xlabel('year')
plt.ylabel('manhattan distance');

plt.subplot(n_rows,2,6)
sns.distplot(wd_model)
mu, std = sp.stats.norm.fit(wd_model)
x = np.linspace(min(wd_model), max(wd_model), 100)
plt.plot(x, sp.stats.norm.pdf(x, mu, std))
plt.xlabel('manhattan distance')
plt.ylabel('probability distribution');
plt.title(f"μ={mu:.2}, σ={std:.2} (model)");

plt.subplot(n_rows,2,7)
sns.scatterplot(x=np.abs(yd), y=neighbors)
slope, intercept, r, p, stderr = sp.stats.linregress(np.abs(yd), neighbors)
x = np.linspace(0, max(yd), 100)
sns.lineplot(x, np.multiply(slope, x) + intercept)
plt.title(f"slope={slope:.2f}; r={r:.2f}; p={p:.1e} (prior)")
plt.xlabel('Δyear')
plt.ylabel('cosine similarity');

plt.subplot(n_rows,2,8)
sns.scatterplot(x=np.abs(yd_model), y=neighbors_model)
slope, intercept, r, p, stderr = sp.stats.linregress(np.abs(yd_model), neighbors_model)
x = np.linspace(0, max(np.abs(yd_model)), 100)
sns.lineplot(x, np.multiply(slope, x) + intercept)
plt.title(f"slope={slope:.2f}; r={r:.2f}; p={p:.1e} (model)")
plt.xlabel('Δyear')
plt.ylabel('cosine similarity');

In [ ]:
plt.figure(figsize=(16,6))

plt.subplot(121)
sns.scatterplot(x='index', y='weight',
                data=pd.DataFrame({'index': model.vectors.indices,
                                   'weight': model.vectors.data}))
plt.ylim([-.1,1.1]);

plt.subplot(122)
plot_distribution(model.vectors.data)

In [ ]:
plt.figure(figsize=(16,6))
plt.subplot(121)
nx.draw_networkx(graph, node_color=['r' if graph.nodes[n]['year']<-500 else 'b'
                                    for n in graph.nodes])
plt.title('original graph')
plt.subplot(122)
nx.draw_networkx(model.graph, node_color=['r' if model.graph.nodes[n]['year']<-500 else 'b'
                                          for n in model.graph.nodes])
plt.title('new graph');

In [ ]:
plt.figure(figsize=(16,6))
sns.distplot([d for _,d in graph.degree], bins=30)
sns.distplot([d for _,d in model.graph.degree], bins=30)
plt.legend(['prior', 'model'])
plt.xlim([-10,110]);

### Discussion

The point of this model is that one can model knowledge discovery as incremental changes on existing knowledge.

The mutation model doesn't monotonically decrease similarity with parent.